In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-128751")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-128751
Azure region: southcentralus
Subscription id: e8f628b3-bb5b-4edf-947a-8637ca6ea7c2
Resource group: aml-quickstarts-128751


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = "training-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": uniform(0.1, 1.0),
    "--max_iter": choice(10, 20, 50, 80, 100)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=".", 
        compute_target=compute_target,
        entry_script="train.py"
        )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
                        estimator=est,
                        hyperparameter_sampling=ps,
                        policy=policy,
                        primary_metric_name = "Accuracy",
                        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                        max_total_runs=20,
                        max_concurrent_runs=4
                    )

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hdr_run = exp.submit(config=hyperdrive_config)
RunDetails(hdr_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [9]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run_hdr = hdr_run.get_best_run_by_primary_metric()
best_run_hdr_metrics = best_run_hdr.get_metrics()

print('Accuracy:', best_run_hdr_metrics['Accuracy'])

# register best model
best_run_hdr.register_model(model_name="best_model_hdr", model_path="outputs/model_hdr.pkl")

Accuracy: 0.9096611026808296


Model(workspace=Workspace.create(name='quick-starts-ws-128751', subscription_id='e8f628b3-bb5b-4edf-947a-8637ca6ea7c2', resource_group='aml-quickstarts-128751'), name=best_model_hdr, id=best_model_hdr:1, version=1, tags={}, properties={})

In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")


In [6]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
pd.concat([x, y], axis=1).to_csv('training/clean_data.csv', index=False)

# upload clean data to datastore
ds = ws.get_default_datastore()
ds.upload(src_dir='training', target_path='training', overwrite=True)

train_data = TabularDatasetFactory.from_delimited_files(path=ds.path('training/clean_data.csv'))

Uploading an estimated of 1 files
Uploading training/clean_data.csv
Uploaded training/clean_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [7]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    compute_target=compute_target,
    primary_metric='accuracy',
    training_data = train_data,
    label_column_name="y",
    n_cross_validations=3)

In [8]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(config=automl_config)
RunDetails(automl_run).show()

Running on remote.


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [11]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run_automl, final_model = automl_run.get_output()
best_run_automl_metrics = best_run_automl.get_metrics()

print('Accuracy:', best_run_automl_metrics['accuracy'])

#print best model from automl
print(final_model)
joblib.dump(final_model, "best_model_automl.pkl")

Accuracy: 0.9165401192648543
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  min_weight_fraction_leaf=0.0,
                                                                                                  n_estimators=50,
                                                                                                  n_jobs=1,
            

['best_model_automl.pkl']

In [12]:
# Clean up compute clusters

try:
    compute_target.delete()
except ComputeTargetException:
    pass

print("Computer cluster deleted")

Computer cluster deleted
Current provisioning state of AmlCompute is "Deleting"

